In [ ]:
pip install spectral mat73  einops

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
import os
import math
import pandas as pd

from einops import rearrange
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from scipy import io
import torch.utils.data
import scipy.io as sio
import mat73
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! ls '/content/drive/MyDrive/A02_RemoteSensingData/UHS_2013_DFTC/'

In [ ]:
#define path
path='/content/drive/MyDrive/A02_RemoteSensingData/UHS_2013_DFTC/'

In [ ]:
# 2.1 Loads Data
# Load hyperpsectral data
hsi_2013_data=sio.loadmat(path+'2013_IEEE_GRSS_DF_Contest_CASI_349_1905_144.mat')['ans']
print('hsi_2013_data shape:', hsi_2013_data.shape)

# Loader Lidar  data
import mat73
lidar_2013_data = sio.loadmat(path+'2013_IEEE_GRSS_DF_Contest_LiDAR.mat')['LiDAR_data']

print('Lidar_2013_data shape:', lidar_2013_data.shape)

#Load ground truth labels
gt_2013_data=sio.loadmat(path+'GRSS2013.mat')['name']
print('gt_2013_data.shape:', gt_2013_data.shape)

In [ ]:
# 2.1 Define the class information
class_info = [(1, "Healthy grass", 'training_sample', 198, 'test_sample', 1053,  'total', 1251),
    (2, "Stressed grass",'training_sample', 190, 'test_sample', 1064,  'total', 1254),
    (3, "Synthetic grass", 'training_sample', 192, 'test_sample', 505,  'total', 697),
    (4, "Trees", 'training_sample', 188, 'test_sample', 1058,  'total', 1244),
    (5, "Soil",'training_sample', 186, 'test_sample', 1056,  'total', 1242),
    (6, "Water", 'training_sample', 182, 'test_sample', 141,  'total', 325),
    (7, "Residential", 'training_sample', 196, 'test_sample', 1072,  'total', 1268),
    (8, "Commercial", 'training_sample', 191, 'test_sample', 1053,  'total', 1244),
    (9, "Road", 'training_sample', 193, 'test_sample', 1059,  'total', 1252),
    (10, "Highway", 'training_sample', 191, 'test_sample', 1036,  'total', 1227),
    (11, "Railway", 'training_sample', 181, 'test_sample', 1054,  'total', 1235),
    (12, "Parking lot 1", 'training_sample', 192, 'test_sample', 1041,  'total', 1233),
    (13, "Parking lot 2", 'training_sample', 184, 'test_sample',285,  'total', 469),
    (14, "Tennis court",'training_sample', 181, 'test_sample', 247,  'total', 428),
    (15, "Running track", 'training_sample', 187, 'test_sample', 473,  'total', 660)]

# Create a dictionary to store class number, class name, and class samples
class_dict = {class_number: {"class_name": class_name,
                             'training_sample': training_sample,
                             'test_sample': test_sample,
                             "total_samples": total}
              for class_number, class_name, _, training_sample, _, test_sample, _, total in class_info}

print(class_dict)

In [ ]:
# 2.2 Samples Extraction

# # Create a mask with all class labels
# mask = np.copy(gt_2013_data)

# # Set the background class to 0
# mask[mask == 0] = 0

# Define patch size and stride
patch_size = 9
stride = 1

# Create an empty list to store patches and labels
hsi_samples = []
lidar_samples = []
labels = []

# Initialize a dictionary to store class count
class_count = {i: 0 for i in class_dict.keys()}

# Function to check if all classes have the required number of samples
def all_classes_completed(class_count, class_dict):
    return all(class_count[class_num] == class_dict[class_num]["total_samples"] for class_num in class_dict.keys())

while not all_classes_completed(class_count, class_dict):
    # Loop through the ground truth data
    for label in class_dict.keys():
        # Get the coordinates of the ground truth pixels
        #coords = np.argwhere((gt_2013_data == label) & (mask > 0))
        coords = np.argwhere(gt_2013_data == label)

        # Shuffle the coordinates to randomize the patch extraction
        np.random.shuffle(coords)

        for coord in coords:
            i, j = coord
            # Calculate the patch indices
            i_start, i_end = i - patch_size // 2, i + patch_size // 2 + 1
            j_start, j_end = j - patch_size // 2, j + patch_size // 2 + 1

            # Check if the indices are within the bounds of the HSI data
            if i_start >= 0 and i_end <= hsi_2013_data.shape[0] and j_start >= 0 and j_end <= hsi_2013_data.shape[1]:
                # Extract the patch
                hsi_patch = hsi_2013_data[i_start:i_end, j_start:j_end, :]

                # Extract the LiDAR patch
                lidar_patch = lidar_2013_data[i_start:i_end, j_start:j_end, :]

                # If the class count is less than the required samples
                if class_count[label] < class_dict[label]["total_samples"]:
                    # Append the patch and its label to the list
                    hsi_samples.append(hsi_patch)
                    lidar_samples.append(lidar_patch)
                    labels.append(label)
                    class_count[label] += 1

                    # If all classes have the required number of samples, exit the loop
                    if all_classes_completed(class_count, class_dict):
                        break

# Convert the list of patches and labels into arrays
hsi_samples = np.array(hsi_samples)
lidar_samples = np.array(lidar_samples)
labels = np.array(labels)
print('hsi_samples shape:', hsi_samples.shape)
print('lidar_samples shape:', lidar_samples.shape)
print('labels shape:', labels.shape)

### Perason Correlation

### SSIM

In [ ]:
# Re-import necessary libraries including SSIM
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from skimage.metrics import structural_similarity as ssim
from sklearn.preprocessing import StandardScaler

# Load hyperspectral data
hsi_2013_data = sio.loadmat(path+'2013_IEEE_GRSS_DF_Contest_CASI_349_1905_144.mat')['ans']
print('hsi_2013_data shape:', hsi_2013_data.shape)

# Load LiDAR data
lidar_2013_data = sio.loadmat(path+'2013_IEEE_GRSS_DF_Contest_LiDAR.mat')['LiDAR_data']

# Reshape data to 2D format (bands x pixels)
hsi_transposed = hsi_2013_data.reshape(hsi_2013_data.shape[2], -1)  # Shape: (144, 349*1905)
lidar_flattened = lidar_2013_data.reshape(1, -1)  # Shape: (1, 349*1905)

# Remove constant bands (zero variance)
non_constant_mask = np.std(hsi_transposed, axis=1) > 1e-6
hsi_filtered = hsi_transposed[non_constant_mask, :]

# Standardize each band separately
scaler = StandardScaler()
hsi_scaled = scaler.fit_transform(hsi_filtered.T).T  # Normalize per band
lidar_scaled = scaler.fit_transform(lidar_flattened.T).T  # Normalize LiDAR

# Remove NaN/Inf values
hsi_scaled = np.nan_to_num(hsi_scaled, nan=0.0)
lidar_scaled = np.nan_to_num(lidar_scaled, nan=0.0)

# Reshape LiDAR to 2D
lidar_2d = lidar_scaled.reshape(349, 1905)

# Calculate SSIM for each HSI band
ssim_scores = []
for i in range(hsi_scaled.shape[0]):
    hsi_band_2d = hsi_scaled[i, :].reshape(349, 1905)
    ssim_val = ssim(hsi_band_2d, lidar_2d, data_range=1.0)
    ssim_scores.append(ssim_val)

# Create results DataFrame
ssim_df = pd.DataFrame({'Houston HSI Band Index': np.arange(1, len(ssim_scores)+1), 'SSIM': ssim_scores})

# Plot results
plt.figure(figsize=(10, 5))
plt.plot(ssim_df["Houston HSI Band Index"], ssim_df["SSIM"], marker='o', linestyle='-')
plt.xlabel("HSI Band Number")
plt.ylabel("SSIM Score")
plt.title("Houston 2013: Structural Similarity (SSIM) Between LiDAR and Each HSI Band")
plt.grid(True)
plt.show()

In [ ]:
#

# Formalisation the code into 3 compoents: 1) Loaddata 2) Train Model 3) Analyse

In [ ]:
# 2.1 Loads Data
# Load hyperpsectral data
hsi_2013_data=sio.loadmat(path+'2013_IEEE_GRSS_DF_Contest_CASI_349_1905_144.mat')['ans']
print('hsi_2013_data shape:', hsi_2013_data.shape)

# Loader Lidar  data
import mat73
lidar_2013_data = sio.loadmat(path+'2013_IEEE_GRSS_DF_Contest_LiDAR.mat')['LiDAR_data']

print('Lidar_2013_data shape:', lidar_2013_data.shape)

#Load ground truth labels
gt_2013_data=sio.loadmat(path+'GRSS2013.mat')['name']
print('gt_2013_data.shape:', gt_2013_data.shape)

# Using PLSR n=1
Now our goal is to compare each HSI band with LiDAR separately, treating:

LiDAR as a single sample (shape: (349, 1905))
Each HSI band as a separate sample (shape: (349, 1905, 144) → (144, 349, 1905) after reshaping)

In [ ]:
# Re-import necessary libraries
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

# Load hyperspectral data
hsi_2013_data=sio.loadmat(path+'2013_IEEE_GRSS_DF_Contest_CASI_349_1905_144.mat')['ans']
print('hsi_2013_data shape:', hsi_2013_data.shape)

# Loader Lidar  data
import mat73
lidar_2013_data = sio.loadmat(path+'2013_IEEE_GRSS_DF_Contest_LiDAR.mat')['LiDAR_data']

# Reshape the data to treat each band as a sample and each pixel as a feature
hsi_transposed = hsi_2013_data.reshape(hsi_2013_data.shape[2], -1)  # Shape: (144, 349*1905)
lidar_flattened = lidar_2013_data.flatten().reshape(1, -1)  # Shape: (1, 349*1905)

# Remove constant bands (zero variance)
non_constant_mask = np.std(hsi_transposed, axis=1) > 1e-6  # Filter out constant bands
hsi_filtered = hsi_transposed[non_constant_mask, :]  # Retain only non-constant bands

# Standardize each band separately
scaler = StandardScaler()
hsi_scaled = scaler.fit_transform(hsi_filtered.T).T  # Normalize per band
lidar_scaled = scaler.fit_transform(lidar_flattened.T).T  # Normalize LiDAR

# Remove NaN or Inf values
hsi_scaled = np.nan_to_num(hsi_scaled, nan=0.0, posinf=0.0, neginf=0.0)
lidar_scaled = np.nan_to_num(lidar_scaled, nan=0.0, posinf=0.0, neginf=0.0)

# Initialize a list to store R² values for each HSI band
r2_values_per_band = []

# Loop through each HSI band and compare it with LiDAR
for i in range(hsi_scaled.shape[0]):
    X_band = hsi_scaled[i, :].reshape(-1, 1)  # Each pixel is a feature (Shape: (664845, 1))
    Y = lidar_scaled.T  # LiDAR as dependent variable (Shape: (664845, 1))

    # Fit PLSR with one component
    pls = PLSRegression(n_components=1)
    pls.fit(X_band, Y)  # Correct shape: (samples, features)

    # Predict LiDAR values
    Y_pred_band = pls.predict(X_band)

    # Compute R² score for this band
    r2_band = r2_score(Y, Y_pred_band)
    r2_values_per_band.append(r2_band)

# Create a DataFrame to store the results
r2_df = pd.DataFrame({'HSI Band': np.arange(1, hsi_scaled.shape[0] + 1), 'R²': r2_values_per_band})

# Plot PLSR correlation (R² scores) for each HSI band
plt.figure(figsize=(10, 5))
plt.plot(r2_df["HSI Band"], r2_df["R²"], marker='o', linestyle='-', label='PLSR Correlation (R²)')
plt.xlabel("HSI Band Number")
plt.ylabel("PLSR Correlation (R²)")
plt.title("PLSR Correlation Between LiDAR and Each HSI Band (Pixels as Features)")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from scipy.ndimage import uniform_filter

# Load hyperspectral data
hsi_2013_data = sio.loadmat(path + '2013_IEEE_GRSS_DF_Contest_CASI_349_1905_144.mat')['ans']
# Load LiDAR data
lidar_2013_data = sio.loadmat(path + '2013_IEEE_GRSS_DF_Contest_LiDAR.mat')['LiDAR_data']

# Reshape HSI to (pixels, bands)
hsi_2d = hsi_2013_data.reshape(-1, hsi_2013_data.shape[2])  # (664545, 144)

# Reshape LiDAR to (pixels, 1)
lidar_2d = lidar_2013_data.reshape(-1, 1)  # (664545, 1)

# Initialize storage for R² scores
num_bands = hsi_2d.shape[1]
r2_scores = np.zeros(num_bands)

# Loop through each HSI band
for band in range(num_bands):
    X = hsi_2d[:, band].reshape(-1, 1)  # Extract 1 band (feature)
    y = lidar_2d  # Target (LiDAR)

    # Fit PLSR (equivalent to linear regression here)
    pls = PLSRegression(n_components=1)
    pls.fit(X, y)

    # Predict and compute R²
    y_pred = pls.predict(X)
    r2_scores[band] = r2_score(y, y_pred)

# Plot R² scores
plt.figure(figsize=(12, 6))
plt.plot(r2_scores, color='blue', linewidth=1)
plt.xlabel('HSI Band Number')
plt.ylabel('R² Score')

# Using PLSR to explorethe relationship between LiDAR and HSI

In [ ]:
import scipy.io as sio
import numpy as np
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# Load hyperspectral data
hsi_2013_data = sio.loadmat(path + '2013_IEEE_GRSS_DF_Contest_CASI_349_1905_144.mat')['ans']
# Load LiDAR data
lidar_2013_data = sio.loadmat(path + '2013_IEEE_GRSS_DF_Contest_LiDAR.mat')['LiDAR_data']

# Reshape the data
hsi_reshaped = hsi_2013_data.reshape(-1, hsi_2013_data.shape[2])  # Reshape to (349*1905, 144)
lidar_reshaped = lidar_2013_data.flatten().reshape(-1, 1)  # Reshape to (349*1905, 1)

# Initialize an empty list to store R² values for each HSI band
r2_values = []

# Loop through each HSI band and compute PLSR with LiDAR data
for i in range(hsi_reshaped.shape[1]):
    X = hsi_reshaped[:, i].reshape(-1, 1)  # Single HSI band as independent variable
    Y = lidar_reshaped  # LiDAR as dependent variable

    # Fit PLSR model
    pls = PLSRegression(n_components=1)
    pls.fit(X, Y)

    # Predict LiDAR values
    Y_pred = pls.predict(X)

    # Compute R² score
    r2 = r2_score(Y, Y_pred)
    r2_values.append(r2)

# Create a DataFrame to store the results
r2_df = pd.DataFrame({'HSI Band': np.arange(1, hsi_reshaped.shape[1] + 1), 'R²': r2_values})
r2_df


import matplotlib.pyplot as plt

# Plot R² values for each HSI band
plt.figure(figsize=(10, 5))
plt.plot(r2_df["HSI Band"], r2_df["R²"], marker='o', linestyle='-', label='R² Score')
plt.xlabel("HSI Band")
plt.ylabel("R² Score")
plt.title("PLSR R² Scores for LiDAR and Each HSI Band")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()


# Spearman Rank Correlation

In [ ]:
# Re-import necessary libraries after execution state reset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import spearmanr

# Load hyperspectral data
hsi_2013_data = sio.loadmat(path + '2013_IEEE_GRSS_DF_Contest_CASI_349_1905_144.mat')['ans']
# Load LiDAR data
lidar_2013_data = sio.loadmat(path + '2013_IEEE_GRSS_DF_Contest_LiDAR.mat')['LiDAR_data']

# Reshape the data
hsi_reshaped = hsi_2013_data.reshape(-1, hsi_2013_data.shape[2])  # Reshape to (349*1905, 144)
lidar_reshaped = lidar_2013_data.flatten().reshape(-1, 1)  # Reshape to (349*1905, 1)

# Initialize an empty list to store Spearman correlation values for each HSI band
spearman_values = []

# Loop through each HSI band and compute Spearman correlation with LiDAR data
for i in range(hsi_reshaped.shape[1]):
    X = hsi_reshaped[:, i].flatten()  # Flatten to 1D array
    Y = lidar_reshaped.flatten()  # Flatten LiDAR data

    # Compute Spearman Rank Correlation
    spearman_corr, _ = spearmanr(X, Y)
    spearman_values.append(spearman_corr)

# Create a DataFrame to store the results
spearman_df = pd.DataFrame({'HSI Band': np.arange(1, hsi_reshaped.shape[1] + 1), 'Spearman Correlation': spearman_values})

# Plot Spearman correlation values for each HSI band
plt.figure(figsize=(10, 5))
plt.plot(spearman_df["HSI Band"], spearman_df["Spearman Correlation"], marker='o', linestyle='-', label='Spearman Correlation')
plt.xlabel("HSI Band")
plt.ylabel("Spearman Correlation")
plt.title("Spearman Correlation Between LiDAR and Each HSI Band")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

# # Display the computed Spearman correlation values
# import ace_tools as tools
# tools.display_dataframe_to_user(name="Spearman Correlation Results", dataframe=spearman_df)


In [ ]:
import scipy.io as sio
import numpy as np
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import StandardScaler

# Path to your data files
path = '/content/drive/MyDrive/A02_RemoteSensingData/UHS_2013_DFTC/'

# Load the data
hsi_2013_data = sio.loadmat(path + '2013_IEEE_GRSS_DF_Contest_CASI_349_1905_144.mat')['ans']
lidar_2013_data = sio.loadmat(path + '2013_IEEE_GRSS_DF_Contest_LiDAR.mat')['LiDAR_data']

# Flatten the spatial dimensions
hsi_reshaped = hsi_2013_data.reshape(-1, hsi_2013_data.shape[2])  # Reshape to (349*1905, 144)
lidar_reshaped = lidar_2013_data.flatten().reshape(-1, 1)  # Reshape to (349*1905, 1)

# Standardize the data
scaler_hsi = StandardScaler()
scaler_lidar = StandardScaler()

hsi_scaled = scaler_hsi.fit_transform(hsi_reshaped)
lidar_scaled = scaler_lidar.fit_transform(lidar_reshaped)

# Fit PLS Regression (1 component, since we analyze each band separately)
pls = PLSRegression(n_components=5)
pls.fit(hsi_scaled, lidar_scaled)

# Extract the regression coefficients for each band
pls_coefficients = pls.coef_.flatten()

# Print the PLS regression coefficients
for i, coef in enumerate(pls_coefficients):
    print(f"Band {i+1} PLS coefficient with LiDAR: {coef}")


In [ ]:
# Bands and corresponding updated PLS coefficients for Houston 2013 dataset
bands_houston = np.arange(1, 145)  # Band numbers from 1 to 144
pls_coefficients_houston = [
    0.33369983225658356, 0.06738072055237548, -0.07588217360298789,
    -0.18777466273264812, -0.19249136846797044, -0.15438531292139338,
    -0.12173441875994985, 0.010870728211409537, 0.04685302460689152,
    0.11285539086322838, 0.16236536135922008, 0.17201420030865197,
    0.16581291781427054, 0.15517050030936277, 0.1507323554992927,
    0.1281906970077688, 0.1106195577757884, 0.09225752035011112,
    0.07939190741856554, 0.06788943487619933, 0.05851101461950048,
    0.05341993248140037, 0.048524986036216994, 0.04085212130374257,
    0.034159873051385764, 0.030551140959256144, 0.030569212029476253,
    0.020560839029031146, 0.010815619861704071, 0.009867677078488537,
    0.0008768752061876678, -0.01506900250833311, -0.024206785338568654,
    -0.037141313915851105, -0.05365951433341847, -0.06869096511807632,
    -0.08073855413089699, -0.0888840596472027, -0.09675874967197821,
    -0.10368268121106688, -0.1100345112499056, -0.11045144982617124,
    -0.10997950749358085, -0.10251371133074112, -0.0935908398221294,
    -0.08365465281996895, -0.07467594149868965, -0.06200038293752526,
    -0.039613401648309926, -0.03032648696215948, -0.02142981535249636,
    -0.014272734303485619, -0.004964119367345583, 0.0034822281348334725,
    0.007230034231577483, 0.012244899193988246, 0.0144238264623406,
    0.013522267422834924, 0.014823402953384558, 0.01742968577268282,
    0.023561972655452377, 0.023517872044269413, 0.02748260758972981,
    0.027118758161786632, 0.02917606084099396, 0.031111343811401602,
    0.030086287467927686, 0.02517092166519472, 0.013312208532618143,
    0.013706090740540277, -0.007047047096054835, -0.021873837568571107,
    -0.039243142263117335, -0.05462920601022978, -0.05377389281451204,
    -0.04411192024427053, -0.04959412832366867, -0.040437767537169475,
    -0.03649641579373868, -0.018854708609204046, 2.080641814931289e-06,
    0.020715361260254368, 0.03540816702618606, 0.05195770495256132,
    0.08298851668583525, 0.0625351879293594, 0.056720716593197364,
    0.056871271543372424, 0.05382641763626919, 0.05006919509124329,
    0.04765126240737959, 0.04299063169737692, 0.03937711312290537,
    0.033403030169157674, 0.033215883947777525, 0.032570751265944727,
    0.02815143653531785, 0.03384960046386779, 0.02954237844528161,
    0.025847807462926916, 0.021550178892990744, 0.01919986446375039,
    0.017876516988347545, 0.01620433469917236, 0.01167748843989462,
    0.008496774328007427, 0.006244021581129349, 0.0016776783570745261,
    -0.002644756759350112, -0.011201630447699956, -0.016232307729178334,
    -0.02131850816583105, -0.02654830273600627, -0.025547911954580615,
    -0.04085600418767381, -0.03540330243233161, -0.0404733239450403,
    -0.0544696947327395, -0.05951066571222288, -0.025470443795617886,
    0.11901281306736823, 0.056698701064272175, 0.05877515052740319,
    0.04481137753008365, 0.021331179563741302, -0.01075832525949212,
    -0.036720941365517595, -0.06112804996262367, -0.06808925468179841,
    -0.060925516080521214, -0.06371258451340442, -0.052098083923520964,
    -0.047942100237201024, -0.043189968385858306, -0.03701064421426775,
    -0.027955105207963547, -0.02282549208562495, -0.01790235939968238,
    -0.011256689365738034, -0.004365015079785364, 0.0036425800641045154,
    0.013439409662024183, 0.017235581739695395, 0.014485877883177962
]

# Plot the updated PLS coefficients for Houston 2013
plt.figure(figsize=(12, 6))
plt.plot(bands_houston, pls_coefficients_houston, marker='o', linestyle='-', color='green', label='Houston 2013 PLS Coefficients')

# Add annotations every 10 bands
for i, txt in enumerate(pls_coefficients_houston):
    if i % 10 == 0:  # Annotate every 10th band
        plt.annotate(f'{txt:.4f}',  # Format the text to 3 decimal places
                     (bands_houston[i], pls_coefficients_houston[i]),
                     textcoords="offset points",  # Positioning the text
                     xytext=(0, 10),  # Distance from text to points (x, y)
                     ha='center')  # Center align text

# Labels and title
plt.xlabel("Houton 2013 HSI Band Number")
plt.ylabel("PLSR Coefficient HSI with LiDAR")
plt.title("Houston 2013: PLS Regression Coefficients (n_components=5)")
plt.axhline(y=0, color='gray', linestyle='--', linewidth=1)  # Reference line at y=0
plt.grid(True)
plt.legend()

# Show the plot
plt.show()



In [ ]:
# Plot the PLS coefficients as a curve line with annotations

plt.figure(figsize=(12, 6))
plt.plot(bands, pls_coefficients, marker='o', linestyle='-', color='b', label='PLS Coefficient')

# Add annotations every 10 bands
for i, txt in enumerate(pls_coefficients):
    if i % 10 == 0:  # Annotate every 10th band
        plt.annotate(f'{txt:.4f}',  # Format the text to 6 decimal places
                     (bands[i], pls_coefficients[i]),
                     textcoords="offset points",  # Positioning the text
                     xytext=(0, 10),  # Distance from text to points (x, y)
                     ha='center')  # Center align text

# Labels and title
plt.xlabel("Houston 2013 HSI Band Number")
plt.ylabel("PLS Coefficient with LiDAR")
plt.title("Relationship between Houston 2013 HSI Bands and LiDAR using PLS Regression")
plt.grid(True)
plt.legend()

# Show the plot
plt.show()


# 1. Computing Pearson Correlation between Lidar and Individual Bands of  HSI

In [ ]:
import scipy.io as sio
import numpy as np
from scipy.stats import pearsonr

# Assuming 'path' is a string with the path to your data files
path='/content/drive/MyDrive/A02_RemoteSensingData/UHS_2013_DFTC/'

# Load the data
hsi_2013_data = sio.loadmat(path + '2013_IEEE_GRSS_DF_Contest_CASI_349_1905_144.mat')['ans']
lidar_2013_data = sio.loadmat(path + '2013_IEEE_GRSS_DF_Contest_LiDAR.mat')['LiDAR_data']

# Flatten the spatial dimensions for correlation calculation
hsi_reshaped = hsi_2013_data.reshape(-1, hsi_2013_data.shape[2])  # Reshape to (349*1905, 144)
lidar_reshaped = lidar_2013_data.flatten()  # Flatten to (349*1905,)

# Calculate the correlation coefficients
correlation_coefficients = []

for i in range(hsi_reshaped.shape[1]):  # Loop over the number of bands
    band_data = hsi_reshaped[:, i]
    correlation, _ = pearsonr(band_data, lidar_reshaped)
    correlation_coefficients.append(correlation)

# # Print the correlation coefficients
# for i, coef in enumerate(correlation_coefficients):
#     print(f"Band {i+1} correlation with LiDAR: {coef}")

# Create a DataFrame to store the results
correlation_df = pd.DataFrame({'HSI Band': np.arange(1, hsi_reshaped.shape[1] + 1), 'Pearson Correlation': correlation_coefficients})

# Plot Pearson correlation values for each HSI band
plt.figure(figsize=(10, 5))
plt.plot(correlation_df["HSI Band"], correlation_df["Pearson Correlation"], marker='o', linestyle='-',color='b', label='Pearson Correlation')
plt.xlabel("HSI Band Number")
plt.ylabel("Correlation Coefficients")
plt.title("Houston 2013: Pearson Correlation Between LiDAR and Each HSI Band")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
import scipy.io as sio
import numpy as np
from scipy.stats import pearsonr

# Assuming 'path' is a string with the path to your data files
path='/content/drive/MyDrive/A02_RemoteSensingData/UHS_2013_DFTC/'

# Load the data
hsi_2013_data = sio.loadmat(path + '2013_IEEE_GRSS_DF_Contest_CASI_349_1905_144.mat')['ans']
lidar_2013_data = sio.loadmat(path + '2013_IEEE_GRSS_DF_Contest_LiDAR.mat')['LiDAR_data']

# Flatten the spatial dimensions for correlation calculation
hsi_reshaped = hsi_2013_data.reshape(-1, hsi_2013_data.shape[2])  # Reshape to (349*1905, 144)
lidar_reshaped = lidar_2013_data.flatten()  # Flatten to (349*1905,)

# Calculate the correlation coefficients
correlation_coefficients = []

for i in range(hsi_reshaped.shape[1]):  # Loop over the number of bands
    band_data = hsi_reshaped[:, i]
    correlation, _ = pearsonr(band_data, lidar_reshaped)
    correlation_coefficients.append(round(correlation, 5))

# The correlation_coefficients list contains all the correlation values rounded to 6 decimal places
print(correlation_coefficients)


In [ ]:
import pandas as pd

# Given correlation coefficients
correlation_coefficients = [0.182271, 0.163957, 0.156177, 0.149681, 0.160716, 0.176037, 0.192657, 0.223383, 0.240639, 0.259519, 0.273894, 0.280391,
                            0.282923, 0.283309, 0.2834, 0.281144, 0.279209, 0.276926, 0.275117, 0.27354, 0.27206, 0.270783, 0.26977, 0.268458,
                            0.26706, 0.265633, 0.264913, 0.262923, 0.261116, 0.259752, 0.257906, 0.255944, 0.254669, 0.253189, 0.251133, 0.249245,
                            0.247297, 0.24538, 0.243225, 0.241082, 0.238912, 0.237007, 0.234834, 0.233103, 0.231315, 0.22966, 0.228002, 0.226928,
                            0.226461, 0.224855, 0.223403, 0.221908, 0.220915, 0.219937, 0.21859, 0.217663, 0.216696, 0.215505, 0.214397, 0.213494,
                            0.213063, 0.211965, 0.211564, 0.210094, 0.209445, 0.208769, 0.207943, 0.2068, 0.205724, 0.205461, 0.204366, 0.204131,
                            0.203459, 0.202858, 0.202248, 0.201999, 0.197749, 0.19368, 0.188035, 0.184516, 0.181369, 0.17891, 0.176703, 0.175359,
                            0.177488, 0.171931, 0.170902, 0.169899, 0.168625, 0.167177, 0.165985, 0.164627, 0.163336, 0.162002, 0.160793, 0.160931,
                            0.159245, 0.158351, 0.157433, 0.156512, 0.155461, 0.154662, 0.153924, 0.153425, 0.152056, 0.151165, 0.15059, 0.149317,
                            0.148277, 0.14709, 0.145947, 0.14446, 0.144117, 0.143418, 0.140975, 0.140798, 0.140118, 0.137706, 0.13675, 0.140609,
                            0.152865, 0.146626, 0.147665, 0.146717, 0.144905, 0.143693, 0.142649, 0.140458, 0.13954, 0.140433, 0.139633,
                            0.139145, 0.139236, 0.139152, 0.139141, 0.139371, 0.139313, 0.139439, 0.139722, 0.140045, 0.1406, 0.141576, 0.143002, 0.145369]

# Create a list of band numbers
band_numbers = list(range(1, len(correlation_coefficients) + 1))

# Create a DataFrame from the two lists
correlation_table = pd.DataFrame({
    'Band Number': band_numbers,
    'Correlation Coefficients with LiDAR': correlation_coefficients
})

# Display the DataFrame
#print(correlation_table)
# Save the DataFrame to a CSV file
correlation_table.to_csv('correlation_coefficients.csv', index=False, float_format='%.6f')

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have already calculated the correlation_coefficients as in the previous code
band_numbers = list(range(1, 145))  # Generate a list of band numbers from 1 to 144

# Plotting
plt.figure(figsize=(9, 4.5))  # Set the figure size
plt.plot(band_numbers, correlation_coefficients, marker='o', linestyle='-', color='b')
plt.title('Correlation Coefficients between LiDAR and HSI Bands')
plt.xlabel('Band Number')
plt.ylabel('Correlation Coefficient with LiDAR')
plt.xticks(range(0, 144, 5))  # Show band numbers with a step of 10 on the x-axis for better readability
plt.grid(True)  # Turn on the grid for easier readability
plt.tight_layout()  # Adjust the plot to ensure everything fits without overlap

# Show the plot
plt.show()



In [ ]:
import matplotlib.pyplot as plt

# Assuming 'band_numbers' and 'correlation_coefficients' are defined as before
plt.figure(figsize=(9,4.5))
plt.plot(band_numbers, correlation_coefficients, marker='o',color='b')

# Annotate every 5th point with its correlation coefficient
for i, txt in enumerate(correlation_coefficients):
    if i % 10 == 0:  # This checks if the band number is divisible by 5
        plt.annotate(f'{txt:.3f}', # this formats the text to 6 decimal places
                     (band_numbers[i], correlation_coefficients[i]),
                     textcoords="offset points", # how to position the text
                     xytext=(0,10), # distance from text to points (x,y)
                     ha='center') # horizontal alignment can be left, right or center

plt.xlabel('Band Number')
plt.ylabel('Correlation Coefficients with LiDAR')
plt.title('Correlation Coefficients for each HSI Band with LiDAR Data at 5 Bands Intervals')
plt.grid(True)
plt.show()



In [ ]:
import matplotlib.pyplot as plt

# Assuming 'band_numbers' and 'correlation_coefficients' are defined as before
plt.figure(figsize=(9,4.5))
plt.plot(band_numbers, correlation_coefficients, marker='o')

# Annotate the first 10 bands and then every 5th point with its correlation coefficient
for i, txt in enumerate(correlation_coefficients):
    if i < 2 or i % 10 == 0:  # Annotate if it's among the first 10 bands or divisible by 5
        plt.annotate(f'{txt:.3f}',
                     (band_numbers[i], correlation_coefficients[i]),
                     textcoords="offset points",
                     xytext=(0,10),
                     ha='center')

plt.xlabel('Band Number')
plt.ylabel('Correlation Coefficients with LiDAR')
plt.title('Correlation Coefficients for each HSI Band with LiDAR Data')
plt.grid(True)
plt.show()


In [ ]:
# The list of band indices you want to highlight, each with a unique color
highlighted_bands_cross = [42, 131, 71, 17, 56, 114, 47, 53, 124, 52, 139, 135, 50, 16, 48, 8, 11, 73, 99, 2]   #
highlighted_bands_lrr =  [67, 143, 45, 51, 50, 49, 48, 47, 46, 44, 53, 43, 42, 41, 39, 38, 37, 52, 54, 35]
highlighted_bands_asps = [2, 5, 7, 16, 1, 47, 34, 42, 10, 38, 49, 15, 18, 17, 13, 19, 20, 14, 9, 12]
highlighted_bands_opbs=[51, 87, 21, 143, 67, 8, 40, 75, 133, 84, 100, 142, 130, 29, 46, 16, 73, 141, 109, 140]
highlighted_bands_trc=[47, 27, 16, 40, 35, 5, 31, 18, 15, 21, 26, 7, 9, 36, 3, 17, 48, 11, 69, 4]
highlighted_bands_bsnet=[118, 127, 40, 105, 90, 18, 70, 82, 8, 143, 115, 1, 22, 75, 80, 51, 21, 103, 113, 26]

In [ ]:
bands_p9= [42, 131, 71, 17, 56, 114, 47, 53, 124, 52, 139, 135, 50, 16, 48, 8, 11, 73, 99, 2, 63, 28, 88, 116, 32, 14, 133, 57, 102, 137, 15, 79, 132, 105, 120, 93, 80, 43, 81, 138, 142, 31, 97, 130, 34, 26, 110, 27, 134, 69]

In [ ]:
bands_asps=[2, 5, 7, 16, 1, 47, 34, 42, 10, 38, 49, 15, 18, 17, 13, 19, 20, 14, 9, 12, 11, 22, 8, 6, 4, 3, 21, 24, 23, 48, 46, 45, 44, 43, 41, 40, 39, 37, 36, 35, 33, 32, 31, 30, 29, 28, 27, 26, 25, 0]

In [ ]:
bands_opbs=[51, 87, 21, 143, 67, 8, 40, 75, 133, 84, 100, 142, 130, 29, 46, 16, 73, 141, 109, 140, 63, 83, 68, 30, 139, 97, 0, 138, 56, 136, 79, 14, 137, 117, 28, 131, 17, 69, 47, 33, 92, 135, 134, 3, 72, 127, 50, 128, 11, 132]

In [ ]:
bands_trc=[47, 27, 16, 40, 35, 5, 31, 18, 15, 21, 26, 7, 9, 36, 3, 17, 48, 11, 69, 4, 33, 32, 75, 68, 70, 142, 72, 73, 74, 143, 76, 77, 78, 79, 80, 81, 82, 66, 67, 55, 65, 64, 49, 50, 51, 52, 53, 54, 84, 56]

In [ ]:
bands_lrr=[67, 143, 45, 51, 50, 49, 48, 47, 46, 44, 53, 43, 42, 41, 39, 38, 37, 52, 54, 35, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 36, 34, 142, 8, 14, 13, 12, 11, 10, 9, 7, 16, 6, 5, 4, 3]

In [ ]:
bands_bsnet=[118, 127, 40, 105, 90, 18, 70, 82, 8, 143, 115, 1, 22, 75, 80, 51, 21, 103, 113, 26, 25, 44, 77, 93, 16, 27, 0, 91, 140, 45, 56, 76, 114, 61, 4, 79, 137, 67, 83, 9, 129, 78, 10, 11, 132, 32, 106, 87, 6, 88]


In [ ]:
len(bands_asps)

In [ ]:
import pandas as pd

# Load the original CSV data
df = pd.read_csv('/content/correlation_coefficients.csv')

# # The list of band indices you want to highlight, each with a unique color
# highlighted_bands_cross = bands_p9

# highlighted_bands_lrr =  bands_lrr
# highlighted_bands_asps = bands_asps
# highlighted_bands_opbs=bands_opbs
# highlighted_bands_trc=bands_trc
# highlighted_bands_bsnet=bands_bsnet

bands_p9 = bands_p9
bands_lrr =  bands_lrr
bands_asps = bands_asps
bands_opbs=bands_opbs
bands_trc=bands_trc
bands_bsnet=bands_bsnet
print('bands_bsnet length:',len(bands_bsnet))
print('bands_asps  length:',len(bands_asps ))
print('bands_opbs length:',len(bands_opbs))
print('bands_trc length:',len(bands_trc))
print('bands_bsnet length:',len(bands_bsnet))
# Define the vertical offset for plotting points that share the same HSI band number

# # Initialize columns for the band selection methods with NaN values
# df['P9'] = pd.Series([float('nan')] * len(df))
# df['ASPS'] = pd.Series([float('nan')] * len(df))
# df['OPBS'] = pd.Series([float('nan')] * len(df))
# df['TRC'] = pd.Series([float('nan')] * len(df))
# df['LRR'] = pd.Series([float('nan')] * len(df))
# df['BSNET'] = pd.Series([float('nan')] * len(df))

# # Initialize columns for the band selection methods with zeros (non-selected)
# df['P9'] = 0
# df['LRR'] = 0
# df['ASPS'] = 0
# df['OPBS'] = 0
# df['TRC'] = 0
# df['BSNET'] = 0

# Check if all band lists have 50 unique values
band_lists = {'P9': bands_p9, 'LRR': bands_lrr, 'ASPS': bands_asps, 'OPBS': bands_opbs, 'TRC': bands_trc, 'BSNET': bands_bsnet}
for name, band_list in band_lists.items():
    unique_bands = set(band_list)
    if len(unique_bands) != 50:
        print(f"Error: The list for {name} has {len(unique_bands)} unique bands. It should have 50.")

# Initialize columns for the band selection methods with zeros (non-selected)
for name in band_lists:
    df[name] = 0

# Update the DataFrame to mark selected bands with 1
for name, band_list in band_lists.items():
    for band in band_list:
        df.loc[df['Band Number'] == band, name] = 1

# Save the updated DataFrame to a new CSV file
df.to_csv('/content/updated_uh_correlation_coefficients.csv', index=False)

In [ ]:
import pandas as pd

# Load the original CSV data
df = pd.read_csv('/content/updated_uh_correlation_coefficients.csv')

# Assuming you have already added the selected band indicators (1 or 0) to the DataFrame
# and the column names for the band selection methods are 'P9', 'ASPS', 'OPBS', 'TRC', 'LRR', 'BSNET'
print(df.head)
# Calculate the sum of selected bands for each method
sum_p9 = df['P9'].sum()
sum_asps = df['ASPS'].sum()
sum_opbs = df['OPBS'].sum()
sum_trc = df['TRC'].sum()
sum_lrr = df['LRR'].sum()
sum_bsnet = df['BSNET'].sum()

# Print out the sums
print(f"Sum of selected bands for P9: {sum_p9}")
print(f"Sum of selected bands for ASPS: {sum_asps}")
print(f"Sum of selected bands for OPBS: {sum_opbs}")
print(f"Sum of selected bands for TRC: {sum_trc}")
print(f"Sum of selected bands for LRR: {sum_lrr}")
print(f"Sum of selected bands for BSNET: {sum_bsnet}")


In [ ]:
# Calculate the weighted sum of correlation coefficients for each method
weighted_sum_p9 = (df['Correlation Coefficients with LiDAR'] * df['P9']).sum()
weighted_sum_lrr = (df['Correlation Coefficients with LiDAR'] * df['LRR']).sum()
weighted_sum_asps = (df['Correlation Coefficients with LiDAR'] * df['ASPS']).sum()
weighted_sum_opbs = (df['Correlation Coefficients with LiDAR'] * df['OPBS']).sum()
weighted_sum_trc = (df['Correlation Coefficients with LiDAR'] * df['TRC']).sum()
weighted_sum_bsnet = (df['Correlation Coefficients with LiDAR'] * df['BSNET']).sum()

# Print out the weighted sums
print(f"Total weighted correlation for P9: {weighted_sum_p9}")
print(f"Total weighted correlation for LRR: {weighted_sum_lrr}")
print(f"Total weighted correlation for ASPS: {weighted_sum_asps}")
print(f"Total weighted correlation for OPBS: {weighted_sum_opbs}")
print(f"Total weighted correlation for TRC: {weighted_sum_trc}")
print(f"Total weighted correlation for BSNET: {weighted_sum_bsnet}")


In [ ]:
import matplotlib.pyplot as plt

# Assuming lidar2_correlations is the list of correlation coefficients you've computed
hsi_band_indices = range(1, len(correlation_coefficients) + 1)
# The list of band indices you want to highlight, each with a unique color
highlighted_bands_cross = bands_p9

highlighted_bands_lrr =  bands_lrr
highlighted_bands_asps = bands_asps
highlighted_bands_opbs=bands_opbs
highlighted_bands_trc=bands_trc
highlighted_bands_bsnet=bands_bsnet
# Define the vertical offset for plotting points that share the same HSI band number
vertical_offset = 0.005

# Define your highlighted bands and their colors
highlighted_bands_info = {
    'CROSS': {'bands': highlighted_bands_cross, 'color': 'blue'},
    'LRR': {'bands': highlighted_bands_lrr, 'color': 'green'},
    'ASPS': {'bands': highlighted_bands_asps, 'color': 'orange'},
    'OPBS': {'bands': highlighted_bands_opbs, 'color': 'purple'},
    'TRC': {'bands': highlighted_bands_trc, 'color': 'cyan'},
    'BSNET': {'bands': highlighted_bands_bsnet, 'color': 'magenta'}
}


# Plotting the correlation with the second LiDAR channel
plt.figure(figsize=(9, 4.5))
plt.plot(hsi_band_indices, correlation_coefficients, linestyle='-', color='grey', alpha=0.5, label='UH2013 HSI LiDAR  Correlation')

# Function to plot highlighted bands with vertical offsets
def plot_highlighted_bands(bands_info):
    for label, info in bands_info.items():
        offsets = np.linspace(-vertical_offset, vertical_offset, len(info['bands']))
        for idx, band in enumerate(sorted(info['bands'])):
            if 0 <= band < len(correlation_coefficients):
                # Adjust index for 1-based HSI band numbering
                adjusted_band_index = band + 1
                # Calculate the correlation coefficient and apply the vertical offset
                adjusted_correlation_value = correlation_coefficients[band] + offsets[idx % len(offsets)]
                # Plot the point with the specified color
                plt.plot(adjusted_band_index, adjusted_correlation_value,
                         marker='o', color=info['color'],
                         label=label if idx == 0 else "")

# Highlight the specified bands with respective colors and labels
plot_highlighted_bands(highlighted_bands_info)

# Add titles and labels
plt.title('UH2013 Correlation Coefficients of HSI Bands with LiDAR')
plt.xlabel('HSI Band Number')
plt.ylabel('Correlation Coefficient')

# Create a custom legend
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.grid(True)
plt.show()

In [ ]:
# Sample lists of highlighted bands for each method (replace with your actual data)

# Dictionary to store the count of points greater than 0.20 for each method
points_count = {
    'CROSS': sum(1 for band in highlighted_bands_cross if correlation_coefficients[band] > 0.21),
    'LRR': sum(1 for band in highlighted_bands_lrr if correlation_coefficients[band] > 0.21),
    'ASPS': sum(1 for band in highlighted_bands_asps if correlation_coefficients[band] > 0.21),
    'OPBS': sum(1 for band in highlighted_bands_opbs if correlation_coefficients[band] > 0.21),
    'TRC': sum(1 for band in highlighted_bands_trc if correlation_coefficients[band] > 0.21),
    'BSNET': sum(1 for band in highlighted_bands_bsnet if correlation_coefficients[band] > 0.21)

}

# Now, print the count for each method
for method, count in points_count.items():
    print(f"{method} contains {count} points with correlation coefficients less than 0.23.")

# 2.0 Trento HSI and LIDAR Correlation

In [ ]:
! ls '/content/drive/MyDrive/A02_RemoteSensingData/TrentoDataSet/'

In [ ]:
# Define the path
path1='/content/drive/MyDrive/A02_RemoteSensingData/TrentoDataSet/'

# Loader HSI TR_map_2018
trento_data=sio.loadmat(path1+'trento_data.mat')

# Extract the HSI_data and LiDAR_data arrays
trento_hsi_data = trento_data['HSI_data']
trento_lidar_data = trento_data['LiDAR_data']

# Reshape the data
trento_lidar_data = np.reshape(trento_lidar_data, (166, 600, 1))

trento_gt=trento_data['ground']
print('trento_hsi_data shape:', trento_hsi_data.shape)
print('trento_lidar_data shape:', trento_lidar_data.shape)
print('trento_gt shape:', trento_gt.shape)

# SSIM Correlation

In [ ]:
# Re-import necessary libraries including SSIM
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from skimage.metrics import structural_similarity as ssim
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

# Define the path
path1='/content/drive/MyDrive/A02_RemoteSensingData/TrentoDataSet/'

# Loader HSI TR_map_2018
trento_data=sio.loadmat(path1+'trento_data.mat')

# Extract the HSI_data and LiDAR_data arrays
trento_hsi_data = trento_data['HSI_data']
trento_lidar_data = trento_data['LiDAR_data']

# Reshape data to 2D format (bands x pixels)
hsi_transposed = trento_hsi_data.reshape(trento_hsi_data.shape[2], -1)  # Reshape to (63, 166*600)
lidar_flattened = trento_lidar_data.reshape(1, -1)  # Shape: (1, 166*600)

# Remove constant bands (zero variance)
non_constant_mask = np.std(hsi_transposed, axis=1) > 1e-6
hsi_filtered = hsi_transposed[non_constant_mask, :]


# Standardize each band separately
scaler = StandardScaler()
hsi_scaled = scaler.fit_transform(hsi_filtered.T).T  # Normalize per band
lidar_scaled = scaler.fit_transform(lidar_flattened.T).T  # Normalize LiDAR

# Remove NaN/Inf values
hsi_scaled = np.nan_to_num(hsi_scaled, nan=0.0)
lidar_scaled = np.nan_to_num(lidar_scaled, nan=0.0)

# Reshape LiDAR to 2D
lidar_2d = lidar_scaled.reshape(166, 600)

# Calculate SSIM for each HSI band
ssim_scores = []
for i in range(hsi_scaled.shape[0]):
    hsi_band_2d = hsi_scaled[i, :].reshape(166, 600)
    ssim_val = ssim(hsi_band_2d, lidar_2d, data_range=1.0)
    ssim_scores.append(ssim_val)

# Create results DataFrame
ssim_df = pd.DataFrame({'Trento HSI Band Index': np.arange(1, len(ssim_scores)+1), 'SSIM': ssim_scores})

# Plot results
plt.figure(figsize=(10, 5))
plt.plot(ssim_df["Trento HSI Band Index"], ssim_df["SSIM"], marker='o', linestyle='-')
plt.xlabel("HSI Band Number")
plt.ylabel("SSIM Score")
plt.title("Trento: Structural Similarity (SSIM) Between LiDAR and Each HSI Band")
plt.grid(True)
plt.show()

### Linear Correlation

In [ ]:
# Re-import necessary libraries
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

# Define the path
path1='/content/drive/MyDrive/A02_RemoteSensingData/TrentoDataSet/'

# Load the Trento dataset
trento_data = sio.loadmat(path1 + 'trento_data.mat')

# Extract the HSI_data and LiDAR_data arrays
trento_hsi_data = trento_data['HSI_data']
trento_lidar_data = trento_data['LiDAR_data']

# Reshape the data to treat each band as a sample and each pixel as a feature
hsi_transposed = trento_hsi_data.reshape(trento_hsi_data.shape[2], -1)  # Shape: (63, 166*600)
lidar_flattened = trento_lidar_data.flatten().reshape(1, -1)  # Shape: (1, 166*600)

# Remove constant bands (zero variance)
non_constant_mask = np.std(hsi_transposed, axis=1) > 1e-6  # Filter out constant bands
hsi_filtered = hsi_transposed[non_constant_mask, :]  # Retain only non-constant bands

# Standardize each band separately
scaler = StandardScaler()
hsi_scaled = scaler.fit_transform(hsi_filtered.T).T  # Normalize per band
lidar_scaled = scaler.fit_transform(lidar_flattened.T).T  # Normalize LiDAR

# # Remove NaN or Inf values
# hsi_scaled = np.nan_to_num(hsi_scaled, nan=0.0, posinf=0.0, neginf=0.0)
# lidar_scaled = np.nan_to_num(lidar_scaled, nan=0.0, posinf=0.0, neginf=0.0)

# Initialize a list to store Pearson correlation values for each HSI band
pearson_values_per_band = []

# Loop through each HSI band and compute Pearson correlation with LiDAR
for i in range(hsi_scaled.shape[0]):
    X_band = hsi_scaled[i, :]  # Each band as a feature (Shape: (99600,))
    Y = lidar_scaled.flatten()  # LiDAR as dependent variable (Shape: (99600,))

    # Compute Pearson correlation
    pearson_corr, _ = pearsonr(X_band, Y)
    pearson_values_per_band.append(pearson_corr)

# Create a DataFrame to store the results
pearson_df = pd.DataFrame({'HSI Band': np.arange(1, hsi_scaled.shape[0] + 1), 'Pearson Correlation': pearson_values_per_band})

# Plot Pearson correlation values for each HSI band
plt.figure(figsize=(10, 5))
plt.plot(pearson_df["HSI Band"], pearson_df["Pearson Correlation"], marker='o', linestyle='-', label='Pearson Correlation')
plt.xlabel("HSI Band Number")
plt.ylabel("Pearson Correlation with LiDAR")
plt.title("Pearson Correlation Between LiDAR and Each HSI Band")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
import scipy.io as sio
import numpy as np
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# Define the path
path1='/content/drive/MyDrive/A02_RemoteSensingData/TrentoDataSet/'

# Loader HSI TR_map_2018
trento_data=sio.loadmat(path1+'trento_data.mat')

# Extract the HSI_data and LiDAR_data arrays
trento_hsi_data = trento_data['HSI_data']
trento_lidar_data = trento_data['LiDAR_data']

# Reshape the data
hsi_reshaped = trento_hsi_data.reshape(-1, trento_hsi_data.shape[2])  # Reshape to (166*600, 63)
lidar_reshaped = trento_lidar_data.flatten().reshape(-1, 1)  # Reshape to (166*600, 1)

# Initialize an empty list to store R² values for each HSI band
r2_values = []

# Loop through each HSI band and compute PLSR with LiDAR data
for i in range(hsi_reshaped.shape[1]):
    X = hsi_reshaped[:, i].reshape(-1, 1)  # Single HSI band as independent variable
    Y = lidar_reshaped  # LiDAR as dependent variable

    # Fit PLSR model
    pls = PLSRegression(n_components=1)
    pls.fit(X, Y)

    # Predict LiDAR values
    Y_pred = pls.predict(X)

    # Compute R² score
    r2 = r2_score(Y, Y_pred)
    r2_values.append(r2)

# Create a DataFrame to store the results
r2_df = pd.DataFrame({'HSI Band': np.arange(1, hsi_reshaped.shape[1] + 1), 'R²': r2_values})
r2_df


import matplotlib.pyplot as plt

# Plot R² values for each HSI band
plt.figure(figsize=(10, 5))
plt.plot(r2_df["HSI Band"], r2_df["R²"], marker='o', linestyle='-', label='R² Score')
plt.xlabel("HSI Band")
plt.ylabel("R² Score")
plt.title("Trento: PLSR R² Scores for LiDAR and Each HSI Band")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

### Spearman Rank Correlation

In [ ]:
# Re-import necessary libraries after execution state reset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import spearmanr

# Define the path
path1='/content/drive/MyDrive/A02_RemoteSensingData/TrentoDataSet/'

# Loader HSI TR_map_2018
trento_data=sio.loadmat(path1+'trento_data.mat')

# Extract the HSI_data and LiDAR_data arrays
trento_hsi_data = trento_data['HSI_data']
trento_lidar_data = trento_data['LiDAR_data']

# Reshape the data
hsi_reshaped = trento_hsi_data.reshape(-1, trento_hsi_data.shape[2])  # Reshape to (166*600, 63)
lidar_reshaped = trento_lidar_data.flatten().reshape(-1, 1)  # Reshape to (166*600, 1)

# Initialize an empty list to store Spearman correlation values for each HSI band
spearman_values = []

# Loop through each HSI band and compute Spearman correlation with LiDAR data
for i in range(hsi_reshaped.shape[1]):
    X = hsi_reshaped[:, i].flatten()  # Flatten to 1D array
    Y = lidar_reshaped.flatten()  # Flatten LiDAR data

    # Compute Spearman Rank Correlation
    spearman_corr, _ = spearmanr(X, Y)
    spearman_values.append(spearman_corr)

# Create a DataFrame to store the results
spearman_df = pd.DataFrame({'HSI Band': np.arange(1, hsi_reshaped.shape[1] + 1), 'Spearman Correlation': spearman_values})

# Plot Spearman correlation values for each HSI band
plt.figure(figsize=(10, 5))
plt.plot(spearman_df["HSI Band"], spearman_df["Spearman Correlation"], marker='o', linestyle='-', label='Spearman Correlation')
plt.xlabel("HSI Band")
plt.ylabel("Spearman Correlation")
plt.title("Spearman Correlation Between LiDAR and Each HSI Band")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

# # Display the computed Spearman correlation values
# import ace_tools as tools
# tools.display_dataframe_to_user(name="Spearman Correlation Results", dataframe=spearman_df)
spearman_df

In [ ]:
# 2.1 Define the class information
class_info = [
    (1, "Apple trees", 129, 3905, 4034),
    (2, "Buildings ", 125, 2778, 2903),
    (3, "Ground", 105, 374, 479),
    (4, "Wood", 154, 8969, 9123),
    (5, "Vineyard", 184, 10317, 10501),  # Corrected this line
    (6, "Roads", 122, 3052, 3174)
]

class_dict = {class_number: {"class_name": class_name, "training": training, "test": test, "samples": samples} for class_number, class_name, training, test, samples in class_info}

# for class_number, class_info in class_dict.items():
#     print(f"Class {class_number}: {class_info['class_name']}")
#     print(f"Training Samples: {class_info['training']}")
#     print(f"Test Samples: {class_info['test']}")
#     print(f"Total Samples: {class_info['samples']}")
#     print()

print(class_dict)

In [ ]:
import scipy.io as sio
import numpy as np
from scipy.stats import pearsonr

# Assuming 'path' is a string with the path to your data files
path1='/content/drive/MyDrive/A02_RemoteSensingData/TrentoDataSet/'

# Loader HSI TR_map_2018
trento_data=sio.loadmat(path1+'trento_data.mat')
#print(trento_data)

In [ ]:
import scipy.io as sio
import numpy as np
from scipy.stats import pearsonr

# Assuming 'path' is a string with the path to your data files
path1='/content/drive/MyDrive/A02_RemoteSensingData/TrentoDataSet/'

# Loader HSI TR_map_2018
trento_data=sio.loadmat(path1+'trento_data.mat')
print()
# Extract the HSI_data and LiDAR_data arrays
trento_hsi_data = trento_data['HSI_data']
#trento_hsi_data=trento_hsi_data.astype(np.uint8)

# Assuming hyperspectral_data is your hyperspectral data with dtype=int32
trento_hsi_data = ((trento_hsi_data- trento_hsi_data.min()) * (255.0 / (trento_hsi_data.max() - trento_hsi_data.min()))).astype(np.uint8)


trento_lidar_data = trento_data['LiDAR_data']
print(' The first band trento_hsi_data:',trento_hsi_data[:,:,0])
print('trento_lidar_data:',trento_lidar_data )
# Reshape the data
trento_lidar_data = np.reshape(trento_lidar_data, (166, 600, 1))

trento_gt=trento_data['ground']
print('trento_hsi_data shape:', trento_hsi_data.shape)
print('trento_lidar_data shape:', trento_lidar_data.shape)


# Flatten the spatial dimensions for correlation calculation
hsi_reshaped = trento_hsi_data.reshape(-1, trento_hsi_data.shape[2])  # Reshape to (166*600, 63)
print('hsi_reshaped  shape:', hsi_reshaped .shape)
lidar_reshaped = trento_lidar_data.flatten()  # Flatten to (166*600,)
print('lidar_reshaped shape:', lidar_reshaped.shape)


# Calculate the correlation coefficients
correlation_coefficients = []

for i in range(hsi_reshaped.shape[1]):  # Loop over the number of bands
    band_data = hsi_reshaped[:, i]
    correlation, _ = pearsonr(band_data, lidar_reshaped)
    correlation_coefficients.append(round(abs(correlation), 5))


# The correlation_coefficients list contains all the correlation values rounded to 6 decimal places
print(correlation_coefficients)




In [ ]:
import pandas as pd

# Given correlation coefficients
correlation_coefficients = [0.21189, 0.1864, 0.17332, 0.15068, 0.14471, 0.1396, 0.13297, 0.13327, 0.13235, 0.13517,
                            0.13557, 0.15419, 0.18779, 0.23515, 0.26057, 0.28654, 0.29547, 0.29409, 0.27343, 0.25299,
                            0.23944, 0.23183, 0.228, 0.21158, 0.19901, 0.19203, 0.17584, 0.15481, 0.14582, 0.13523, 0.13968,
                            0.17544, 0.27224, 0.37036, 0.39995, 0.35292, 0.30521, 0.27791, 0.27326, 0.27504, 0.2639, 0.2655,
                            0.26678, 0.26814, 0.28096, 0.28169, 0.27789, 0.27459, 0.27685, 0.27641, 0.27685, 0.28019,
                            0.28799, 0.29017, 0.29206, 0.28847, 0.30574, 0.27414, 0.26464, 0.26984, 0.28525, 0.27979, 0.28513]

# Create a list of band numbers
band_numbers = list(range(1, len(correlation_coefficients) + 1))

# Create a DataFrame from the two lists
correlation_table = pd.DataFrame({
    'Band Number': band_numbers,
    'Correlation Coefficients with LiDAR': correlation_coefficients
})

# Display the DataFrame
#print(correlation_table)
# Save the DataFrame to a CSV file
correlation_table.to_csv('correlation_coefficients_Trento.csv', index=False, float_format='%.6f')

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have already calculated the correlation_coefficients as in the previous code
band_numbers = list(range(1, 64))  # Generate a list of band numbers from 1 to 144

# Plotting
plt.figure(figsize=(8, 4))  # Set the figure size
plt.plot(band_numbers, correlation_coefficients, marker='o', linestyle='-', color='b')
plt.title('Trento: Pearson Correlation Coefficients between LiDAR and Each HSI Band')
plt.xlabel('Band Number')
plt.ylabel('Correlation Coeficients ')
plt.xticks(range(0, 64, 5))  # Show band numbers with a step of 10 on the x-axis for better readability
plt.grid(True)  # Turn on the grid for easier readability
plt.tight_layout()  # Adjust the plot to ensure everything fits without overlap

# Show the plot
plt.show()



In [ ]:
import matplotlib.pyplot as plt

# Assuming 'band_numbers' and 'correlation_coefficients' are defined as before
plt.figure(figsize=(9,4.5))
plt.plot(band_numbers, correlation_coefficients, marker='o',linestyle='-', color='b')

# Annotate the first 10 bands and then every 5th point with its correlation coefficient
for i, txt in enumerate(correlation_coefficients):
    if i < 2 or i % 10 == 0:  # Annotate if it's among the first 10 bands or divisible by 5
        plt.annotate(f'{txt:.4f}',
                     (band_numbers[i], correlation_coefficients[i]),
                     textcoords="offset points",
                     xytext=(0,10),
                     ha='center')

plt.xlabel('Band Number')
plt.ylabel('Correlation Coefficients with LiDAR')
plt.title('Trento Correlation Coefficients for each HSI Band with LiDAR Data')
plt.grid(True)
plt.show()


In [ ]:
# The list of band indices you want to highlight, each with a unique color
highlighted_bands_cross = [37, 52, 39, 50, 46, 42, 31, 35, 18, 34, 43, 26, 14, 25, 59, 30, 44, 22, 57, 21]
highlighted_bands_lrr =  [34, 33, 9, 35, 7, 8, 10, 6, 18, 32, 11, 5, 62, 4, 61, 49, 48, 60, 19, 20]
highlighted_bands_asps = [16, 5, 7, 49, 13, 22, 21, 20, 19, 18, 17, 15, 14, 11, 12, 48, 10, 9, 8,6]
highlighted_bands_opbs=[29, 39, 0, 58, 57, 59, 1, 2, 62, 61, 60, 3, 56, 4, 16, 5, 6, 55, 54, 7]
highlighted_bands_trc=[34, 18, 33, 35, 17, 9, 7, 10, 8, 6, 47, 32, 43, 11, 44, 46, 4, 28, 30, 3]
highlighted_bands_bsnet=[45, 33, 8, 38, 46, 16, 57, 58, 62, 40, 19, 13, 2, 12, 42, 52, 35, 25, 9, 3]

In [ ]:
#Ourmethod
# Patch_size= 9 band selection
bands_p9=[37, 52, 39, 50, 46, 42, 31, 35, 18, 34, 43, 26, 14, 25, 59, 30, 44, 22, 57, 21, 5, 32, 6, 58, 27, 38, 36, 4, 49, 8, 56, 20, 12, 55, 0, 61, 11, 16, 54, 53, 3, 17, 51, 7, 33, 29, 41, 9, 24, 40, 1, 10, 23, 45, 47, 15, 28, 2, 13, 62, 48, 60, 19]

In [ ]:
#LRR
bands_lrr=[34, 33, 9, 35, 7, 8, 10, 6, 18, 32, 11, 5, 62, 4, 61, 49, 48, 60, 19, 20, 12, 22, 23, 59, 57, 27, 28, 30, 17, 3, 47, 52, 51, 37, 38, 16, 40, 50, 43, 44, 46, 45, 56, 55, 58, 2, 0, 1, 29, 31]

In [ ]:
bands_asps=[16, 5, 7, 49, 13, 22, 21, 20, 19, 18, 17, 15, 14, 11, 12, 48, 10, 9, 8, 6, 4, 3, 2, 1, 23, 24, 25, 26, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 0]

In [ ]:
#opbs
bands_opbs=[29, 39, 0, 58, 57, 59, 1, 2, 62, 61, 60, 3, 56, 4, 16, 5, 6, 55, 54, 7, 33, 30, 53, 8, 31, 9, 52, 10, 28, 51, 11, 25, 32, 12, 27, 50, 26, 13, 49, 23, 38, 24, 48, 21, 45, 34, 46, 20, 47, 22]

In [ ]:
#patch_size=9 obtianed on 09252023
bands_trc=[34, 18, 33, 35, 17, 9, 7, 10, 8, 6, 47, 32, 43, 11, 44, 46, 4, 28, 30, 3, 5, 54, 57, 60, 61, 59, 58, 62, 56, 55, 53, 52, 51, 50, 27, 2, 1, 49, 0, 40, 42, 21, 31, 22, 45, 48, 29, 19, 20, 13]

In [ ]:
#BSNET
#Trento Top 50
bands_bsnet=[45, 33, 8, 38, 46, 16, 57, 58, 62, 40, 19, 13, 2, 12, 42, 52, 35, 25, 9, 3, 34, 27, 50, 11, 28, 0, 37, 7, 54, 17, 48, 53, 4, 29, 21, 24, 22, 49, 1, 60, 32, 31, 36, 23, 47, 18, 15, 6, 56, 26]

In [ ]:
# The list of band indices you want to highlight, each with a unique color
highlighted_bands_cross = bands_p9
highlighted_bands_lrr =  bands_lrr
highlighted_bands_asps = bands_asps
highlighted_bands_opbs=bands_opbs
highlighted_bands_trc=bands_trc
highlighted_bands_bsnet=bands_bsnet

In [ ]:
import matplotlib.pyplot as plt

# Assuming lidar2_correlations is the list of correlation coefficients you've computed
hsi_band_indices = range(1, len(correlation_coefficients) + 1)

# Define the vertical offset for plotting points that share the same HSI band number
vertical_offset = 0.005

# Define your highlighted bands and their colors
highlighted_bands_info = {
    'CROSS': {'bands': highlighted_bands_cross, 'color': 'blue'},
    'LRR': {'bands': highlighted_bands_lrr, 'color': 'green'},
    'ASPS': {'bands': highlighted_bands_asps, 'color': 'orange'},
    'OPBS': {'bands': highlighted_bands_opbs, 'color': 'purple'},
    'TRC': {'bands': highlighted_bands_trc, 'color': 'cyan'},
    'BSNET': {'bands': highlighted_bands_bsnet, 'color': 'magenta'}
}

# Plotting the correlation with the second LiDAR channel
plt.figure(figsize=(9, 4.5))
plt.plot(hsi_band_indices, correlation_coefficients, linestyle='-', color='grey', alpha=0.5, label='Trento HSI LiDAR  Correlation')

# Function to plot highlighted bands with vertical offsets
def plot_highlighted_bands(bands_info):
    for label, info in bands_info.items():
        offsets = np.linspace(-vertical_offset, vertical_offset, len(info['bands']))
        for idx, band in enumerate(info['bands']):
            if band < len(correlation_coefficients):
                plt.plot(band + 1, correlation_coefficients[band] + offsets[idx],
                         marker='o', color=info['color'], label=label if idx == 0 else "")

# Highlight the specified bands with respective colors and labels
plot_highlighted_bands(highlighted_bands_info)

# Add titles and labels
plt.title('Trento Correlation Coefficients of HSI Bands with LiDAR')
plt.xlabel('HSI Band Number')
plt.ylabel('Correlation Coefficient')

# Create a custom legend
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.grid(True)
plt.show()

In [ ]:
# Sample lists of highlighted bands for each method (replace with your actual data)

# The list of band indices you want to highlight, each with a unique color
highlighted_bands_cross = bands_p9[:50]
highlighted_bands_lrr =  bands_lrr
highlighted_bands_asps = bands_asps
highlighted_bands_opbs=bands_opbs
highlighted_bands_trc=bands_trc
highlighted_bands_bsnet=bands_bsnet
print("length of P9:", len(highlighted_bands_cross))

# Dictionary to store the count of points greater than 0.20 for each method
points_count = {
    'CROSS': sum(1 for band in highlighted_bands_cross if correlation_coefficients[band] > 0.238),
    'LRR': sum(1 for band in highlighted_bands_lrr if correlation_coefficients[band] > 0.238),
    'ASPS': sum(1 for band in highlighted_bands_asps if correlation_coefficients[band] > 0.238),
    'OPBS': sum(1 for band in highlighted_bands_opbs if correlation_coefficients[band] > 0.238),
    'TRC': sum(1 for band in highlighted_bands_trc if correlation_coefficients[band] > 0.238),
    'BSNET': sum(1 for band in highlighted_bands_bsnet if correlation_coefficients[band] > 0.238)

}

# Now, print the count for each method
for method, count in points_count.items():
    print(f"{method} contains {count} points with correlation coefficients less than than 0.25.")

##3. MUUFL LiDAR and HSI COrrelation

In [ ]:
! ls '/content/drive/MyDrive/A02_RemoteSensingData/MUUFLGUlfportDataCollection/'

In [ ]:
# 2.1 Define the class information
class_info = [
    (1, "Tree", 150, 23246, 23396),
    (2, "Mostly grass", 150, 4270,4420),
    (3, "Mixed ground surface", 150, 6882, 7032),
    (4, "Dirt and sand", 150, 1826, 1976),
    (5, "Road", 150, 6687, 6837),
    (6, "Water", 150, 466, 616),
    (7, "Building shadow", 150, 2233, 2383),
    (8, "Building", 150, 6240, 6390),
    (9, "Sidewalk", 150, 1385, 1535),
    (10, "Yellow curb", 150, 183, 333),
    (11, "Cloth panels", 150, 269, 419)

]

class_dict = {class_number: {"class_name": class_name, "training": training, "test": test, "samples": samples} for class_number, class_name, training, test, samples in class_info}

# for class_number, class_info in class_dict.items():
#     print(f"Class {class_number}: {class_info['class_name']}")
#     print(f"Training Samples: {class_info['training']}")
#     print(f"Test Samples: {class_info['test']}")
#     print(f"Total Samples: {class_info['samples']}")
#     print()


#print(class_dict)

In [ ]:
# path
path ='/content/drive/MyDrive/A02_RemoteSensingData/MUUFLGUlfportDataCollection/'
hsi_data_campus3=sio.loadmat( path + "muufl_gulfport_campus_3.mat")
hsi_data_campus4=sio.loadmat( path + "muufl_gulfport_campus_4.mat")
lidar_data= sio.loadmat(path + "muufl_gulfport_campus_1_hsi_220_label.mat")
gt_data=sio.loadmat(path + "MUUFL_TruthForSubImage.mat")

In [ ]:
import scipy.io
import numpy as np

# Load the MATLAB file
mat = sio.loadmat(path+"muufl_gulfport_campus_1_hsi_220_label.mat")

# Access the hyperspectral data
hsi_data = mat['hsi']

# Depending on the structure of the .mat file, you might have to access nested structures
# For example, if the data is nested within a structure or cell array in MATLAB, you might need to do something like this:
hsi_data = mat['hsi'][0,0]['Data']
print('hsi_data .shape:',hsi_data .shape)

# Access Ground truuth

hsi = ((mat['hsi'])[0])[0]
# RGB Image
rgbIm = hsi[-1]

# Ground truth
truth = ((hsi[-2])[0])[-1]
truth = truth[-1]
print('truth.shape:', truth.shape)

# Access LiDAR
lidar = ((((hsi[-4])[0])[0])[0])[0]

# x, y, z. z contains Height and Intensity
x, y, z, info = lidar[0], lidar[1], lidar[2], lidar[3]

print('x.shape:', x.shape)
print('y.shape:', y.shape)
print('z.shape:', z.shape)


In [ ]:
muufl_hsi_data=hsi_data
muufl_lidar_data=z
print('muufl_hsi_data:',muufl_hsi_data.shape)
print('muufl_lidar_data:',muufl_lidar_data.shape)

# SSIM

In [ ]:
# Re-import necessary libraries including SSIM
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from skimage.metrics import structural_similarity as ssim


#####
muufl_hsi_data=hsi_data
muufl_lidar_data=z
print('muufl_hsi_data:',muufl_hsi_data.shape)
print('muufl_lidar_data:',muufl_lidar_data.shape)

# Ensure the HSI and LiDAR data are correctly extracted
muufl_hsi_data = hsi_data.astype(np.float32)  # Convert HSI data to float precision
muufl_lidar_data = muufl_lidar_data[:, :, 0]  # Use the first LiDAR channel

# Reshape the data to treat each band as a sample and each pixel as a feature
hsi_transposed = muufl_hsi_data.reshape(muufl_hsi_data.shape[2], -1)  # Shape: (64, 325*220)
lidar_flattened = muufl_lidar_data.flatten().reshape(1, -1)  # Shape: (1, 325*220)
####


# Remove constant bands (zero variance)
non_constant_mask = np.std(hsi_transposed, axis=1) > 1e-6
hsi_filtered = hsi_transposed[non_constant_mask, :]

# Normalize to [0,1] using MinMaxScaler
scaler = MinMaxScaler()
hsi_scaled = scaler.fit_transform(hsi_filtered.T).T  # Normalize each HSI band
lidar_scaled = scaler.fit_transform(lidar_flattened.T).T  # Normalize LiDAR

# Remove NaN/Inf values
hsi_scaled = np.nan_to_num(hsi_scaled, nan=0.0)
lidar_scaled = np.nan_to_num(lidar_scaled, nan=0.0)

# Reshape LiDAR to 2D
lidar_2d = lidar_scaled.reshape(325, 220)

# Calculate SSIM for each HSI band
ssim_scores = []
for i in range(hsi_scaled.shape[0]):
    hsi_band_2d = hsi_scaled[i, :].reshape(325, 220)
    ssim_val = ssim(hsi_band_2d, lidar_2d, data_range=1.0)
    ssim_scores.append(ssim_val)

# Create results DataFrame
ssim_df = pd.DataFrame({'MUFFL HSI Band Index': np.arange(1, len(ssim_scores)+1), 'SSIM': ssim_scores})

# Plot results
plt.figure(figsize=(10, 5))
plt.plot(ssim_df["MUFFL HSI Band Index"], ssim_df["SSIM"], marker='o', linestyle='-')
plt.xlabel("HSI Band Number")
plt.ylabel("SSIM Score")
plt.title("MUFFL: Structural Similarity (SSIM) Between LiDAR and Each HSI Band")
plt.grid(True)
plt.show()

## PLSR Linear

In [ ]:
# Re-import necessary libraries
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

muufl_hsi_data=hsi_data
muufl_lidar_data=z
print('muufl_hsi_data:',muufl_hsi_data.shape)
print('muufl_lidar_data:',muufl_lidar_data.shape)

# Ensure the HSI and LiDAR data are correctly extracted
muufl_hsi_data = hsi_data.astype(np.float32)  # Convert HSI data to float precision
muufl_lidar_data = muufl_lidar_data[:, :, 0]  # Use the first LiDAR channel

# Reshape the data to treat each band as a sample and each pixel as a feature
hsi_transposed = muufl_hsi_data.reshape(muufl_hsi_data.shape[2], -1)  # Shape: (144, 349*1905)
lidar_flattened = muufl_lidar_data.flatten().reshape(1, -1)  # Shape: (1, 349*1905)

# Remove constant bands (zero variance)
non_constant_mask = np.std(hsi_transposed, axis=1) > 1e-6  # Filter out constant bands
hsi_filtered = hsi_transposed[non_constant_mask, :]  # Retain only non-constant bands

# Standardize each band separately
scaler = StandardScaler()
hsi_scaled = scaler.fit_transform(hsi_filtered.T).T  # Normalize per band
lidar_scaled = scaler.fit_transform(lidar_flattened.T).T  # Normalize LiDAR

# Remove NaN or Inf values
hsi_scaled = np.nan_to_num(hsi_scaled, nan=0.0, posinf=0.0, neginf=0.0)
lidar_scaled = np.nan_to_num(lidar_scaled, nan=0.0, posinf=0.0, neginf=0.0)

# Initialize a list to store R² values for each HSI band
r2_values_per_band = []

# Loop through each HSI band and compare it with LiDAR
for i in range(hsi_scaled.shape[0]):
    X_band = hsi_scaled[i, :].reshape(-1, 1)  # Each pixel is a feature (Shape: (664845, 1))
    Y = lidar_scaled.T  # LiDAR as dependent variable (Shape: (664845, 1))

    # Fit PLSR with one component
    pls = PLSRegression(n_components=1)
    pls.fit(X_band, Y)  # Correct shape: (samples, features)

    # Predict LiDAR values
    Y_pred_band = pls.predict(X_band)

    # Compute R² score for this band
    r2_band = r2_score(Y, Y_pred_band)
    r2_values_per_band.append(r2_band)

# Create a DataFrame to store the results
r2_df = pd.DataFrame({'HSI Band': np.arange(1, hsi_scaled.shape[0] + 1), 'R²': r2_values_per_band})

# Plot PLSR correlation (R² scores) for each HSI band
plt.figure(figsize=(10, 5))
plt.plot(r2_df["HSI Band"], r2_df["R²"], marker='o', linestyle='-', label='PLSR Correlation (R²)')
plt.xlabel("HSI Band Number")
plt.ylabel("PLSR Correlation (R²)")
plt.title("PLSR Correlation Between LiDAR and Each HSI Band (Pixels as Features)")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()


### PLSR Lidar and HSI correlation

In [ ]:
import scipy.io as sio
import numpy as np
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

muufl_hsi_data=hsi_data
muufl_lidar_data=z
print('muufl_hsi_data:',muufl_hsi_data.shape)
print('muufl_lidar_data:',muufl_lidar_data.shape)

# Ensure the HSI and LiDAR data are correctly extracted
muufl_hsi_data = hsi_data.astype(np.float32)  # Convert HSI data to float precision
muufl_lidar_data = muufl_lidar_data[:, :, 0]  # Use the first LiDAR channel

# Reshape HSI and LiDAR data
hsi_reshaped = muufl_hsi_data.reshape(-1, muufl_hsi_data.shape[2])  # (325*220, 64)
lidar_reshaped = muufl_lidar_data.flatten().reshape(-1, 1)  # (325*220, 1)

# Initialize an empty list to store R² values for each HSI band
r2_values = []

# Loop through each HSI band and compute PLSR with LiDAR data
for i in range(hsi_reshaped.shape[1]):
    X = hsi_reshaped[:, i].reshape(-1, 1)  # Single HSI band as independent variable
    Y = lidar_reshaped  # LiDAR as dependent variable

    # Fit PLSR model
    pls = PLSRegression(n_components=1)
    pls.fit(X, Y)

    # Predict LiDAR values
    Y_pred = pls.predict(X)

    # Compute R² score
    r2 = r2_score(Y, Y_pred)
    r2_values.append(r2)

# Create a DataFrame to store the results
r2_df = pd.DataFrame({'HSI Band': np.arange(1, hsi_reshaped.shape[1] + 1), 'R²': r2_values})
r2_df


import matplotlib.pyplot as plt

# Plot R² values for each HSI band
plt.figure(figsize=(10, 5))
plt.plot(r2_df["HSI Band"], r2_df["R²"], marker='o', linestyle='-', label='R² Score')
plt.xlabel("HSI Band")
plt.ylabel("R² Score")
plt.title("MUFFL: PLSR R² Scores for LiDAR and Each HSI Band")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

### Spearman Rank Correlation

In [ ]:
# Re-import necessary libraries after execution state reset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import spearmanr

muufl_hsi_data=hsi_data
muufl_lidar_data=z
print('muufl_hsi_data:',muufl_hsi_data.shape)
print('muufl_lidar_data:',muufl_lidar_data.shape)

# Ensure the HSI and LiDAR data are correctly extracted
muufl_hsi_data = hsi_data.astype(np.float32)  # Convert HSI data to float precision
muufl_lidar_data = muufl_lidar_data[:, :, 0]  # Use the first LiDAR channel

# Reshape HSI and LiDAR data
hsi_reshaped = muufl_hsi_data.reshape(-1, muufl_hsi_data.shape[2])  # (325*220, 64)
lidar_reshaped = muufl_lidar_data.flatten().reshape(-1, 1)  # (325*220, 1)band
spearman_values = []

# Loop through each HSI band and compute Spearman correlation with LiDAR data
for i in range(hsi_reshaped.shape[1]):
    X = hsi_reshaped[:, i].flatten()  # Flatten to 1D array
    Y = lidar_reshaped.flatten()  # Flatten LiDAR data

    # Compute Spearman Rank Correlation
    spearman_corr, _ = spearmanr(X, Y)
    spearman_values.append(spearman_corr)

# Create a DataFrame to store the results
spearman_df = pd.DataFrame({'HSI Band': np.arange(1, hsi_reshaped.shape[1] + 1), 'Spearman Correlation': spearman_values})

# Plot Spearman correlation values for each HSI band
plt.figure(figsize=(10, 5))
plt.plot(spearman_df["HSI Band"], spearman_df["Spearman Correlation"], marker='o', linestyle='-', label='Spearman Correlation')
plt.xlabel("HSI Band")
plt.ylabel("Spearman Correlation")
plt.title("Spearman Correlation Between LiDAR and Each HSI Band")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()


In [ ]:
# Identify the most complementary bands based on strongest Spearman correlations

# Sort the bands based on absolute Spearman correlation values (both positive and negative)
top_complementary_bands = spearman_df.reindex(spearman_df["Spearman Correlation"].abs().sort_values(ascending=False).index)

# Select top 5 most strongly correlated bands (both positive and negative correlations)
top_n = 10
selected_bands = top_complementary_bands.head(top_n)

# # Display the top complementary bands
# import ace_tools as tools
# tools.display_dataframe_to_user(name="Top Complementary HSI Bands", dataframe=selected_bands)

# Visualizing the relationships between LiDAR and selected HSI bands
plt.figure(figsize=(12, 6))

for i, band in enumerate(selected_bands["HSI Band"]):
    plt.scatter(hsi_reshaped[:, band - 1], lidar_reshaped, alpha=0.3, label=f'HSI Band {band}')

plt.xlabel("HSI Band Reflectance")
plt.ylabel("LiDAR Value")
plt.title("Scatter Plot of LiDAR vs. Selected HSI Bands")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()


In [ ]:
pip install tools

In [ ]:
# Re-import necessary libraries after execution state reset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
import tools

# Ensure the HSI and LiDAR data are correctly extracted
muufl_hsi_data = hsi_reshaped.astype(np.float32)  # Convert HSI data to float precision
muufl_lidar_data = lidar_reshaped.flatten()  # Flatten LiDAR data

# Compute Spearman correlation for each HSI band
spearman_values = []

for i in range(muufl_hsi_data.shape[1]):
    X = muufl_hsi_data[:, i].flatten()  # Flatten to 1D array
    Y = muufl_lidar_data.flatten()  # Flatten LiDAR data

    # Compute Spearman Rank Correlation
    spearman_corr, _ = spearmanr(X, Y)
    spearman_values.append(spearman_corr)

# Create a DataFrame to store the results
spearman_df = pd.DataFrame({'HSI Band': np.arange(1, muufl_hsi_data.shape[1] + 1), 'Spearman Correlation': spearman_values})

# Identify the most complementary bands based on strongest Spearman correlations
top_complementary_bands = spearman_df.reindex(spearman_df["Spearman Correlation"].abs().sort_values(ascending=False).index)

# Select top 10 most strongly correlated bands (both positive and negative correlations)
top_n = 10
selected_bands = top_complementary_bands.head(top_n)

# # Display the top complementary bands
# import tools
# tools.display_dataframe_to_user(name="Top Complementary HSI Bands", dataframe=selected_bands)

# Visualizing the relationships between LiDAR and selected HSI bands
plt.figure(figsize=(12, 6))

for i, band in enumerate(selected_bands["HSI Band"]):
    plt.scatter(muufl_hsi_data[:, band - 1], muufl_lidar_data, alpha=0.3, label=f'HSI Band {band}')

plt.xlabel("HSI Band Reflectance")
plt.ylabel("LiDAR Value")
plt.title("Scatter Plot of LiDAR vs. Selected HSI Bands")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

# Plot Spearman correlation values for each HSI band
plt.figure(figsize=(10, 5))
plt.plot(spearman_df["HSI Band"], spearman_df["Spearman Correlation"], marker='o', linestyle='-', label='Spearman Correlation')
plt.xlabel("HSI Band")
plt.ylabel("Spearman Correlation")
plt.title("Spearman Correlation Between LiDAR and Each HSI Band")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()


In [ ]:
selected_bands

In [ ]:
# Re-plot scatter plots with transposed axes (swap X and Y axes)

plt.figure(figsize=(12, 6))

for i, band in enumerate(selected_bands["HSI Band"]):
    plt.scatter(lidar_reshaped, muufl_hsi_data[:, band - 1], alpha=0.3, label=f'HSI Band {band}')

plt.xlabel("LiDAR Value")  # Swapped axes
plt.ylabel("HSI Band Reflectance")  # Swapped axes
plt.title("Scatter Plot of Selected HSI Bands vs. LiDAR (Transposed Axes)")
plt.legend()
plt.grid(True)

# Show the corrected plot
plt.show()


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression

muufl_hsi_data=hsi_data
muufl_lidar_data=z
print('muufl_hsi_data:',muufl_hsi_data.shape)
print('muufl_lidar_data:',muufl_lidar_data.shape)

# Ensure the HSI and LiDAR data are correctly extracted
muufl_hsi_data = hsi_data.astype(np.float32)  # Convert HSI data to float precision
muufl_lidar_data = muufl_lidar_data[:, :, 0]  # Use the first LiDAR channel

# Reshape HSI and LiDAR data
muufl_hsi_reshaped = muufl_hsi_data.reshape(-1, muufl_hsi_data.shape[2])  # (325*220, 64)
muufl_lidar_reshaped = muufl_lidar_data.flatten().reshape(-1, 1)  # (325*220, 1)

# Standardize the data
scaler_hsi = StandardScaler()
scaler_lidar = StandardScaler()

hsi_scaled = scaler_hsi.fit_transform(muufl_hsi_reshaped)
lidar_scaled = scaler_lidar.fit_transform(muufl_lidar_reshaped)

# Fit PLS Regression with 5 components
pls_muufl = PLSRegression(n_components=5)
pls_muufl.fit(hsi_scaled, lidar_scaled)

# Extract PLS regression coefficients for each band
pls_coefficients_muufl = pls_muufl.coef_.flatten()

# Prepare band numbers
bands_muufl = np.arange(1, 65)  # Band numbers from 1 to 64

# Ensure the coefficient array matches the band count
if len(pls_coefficients_muufl) > len(bands_muufl):
    pls_coefficients_muufl = pls_coefficients_muufl[:len(bands_muufl)]
elif len(pls_coefficients_muufl) < len(bands_muufl):
    bands_muufl = bands_muufl[:len(pls_coefficients_muufl)]

# Plot the updated PLS coefficients for MUUFL dataset
plt.figure(figsize=(12, 6))
plt.plot(bands_muufl, pls_coefficients_muufl, marker='o', linestyle='-', color='teal', label='MUUFL PLS Coefficients')

# Add annotations every 10 bands
for i, txt in enumerate(pls_coefficients_muufl):
    if i % 10 == 0:  # Annotate every 10th band
        plt.annotate(f'{txt:.4f}',  # Format the text to 3 decimal places
                     (bands_muufl[i], pls_coefficients_muufl[i]),
                     textcoords="offset points",  # Positioning the text
                     xytext=(0, 10),  # Distance from text to points (x, y)
                     ha='center')  # Center align text

# Labels and title
plt.xlabel("MUFFL HSI Band Number")
plt.ylabel("PLSR Coefficient HSI Band with LiDAR")
plt.title("MUFFL Data: PLS Regression Coefficients (n_components=5)")
plt.axhline(y=0, color='gray', linestyle='--', linewidth=1)  # Reference line at y=0
plt.grid(True)
plt.legend()

# Show the plot
plt.show()



In [ ]:
# Ensure the HSI and LiDAR data are correctly extracted
muufl_hsi_data = hsi_data.astype(np.float32)  # Convert HSI data to float precision
muufl_lidar_data = muufl_lidar_data[:, :, 0]  # Use the first LiDAR channel

# Reshape HSI and LiDAR data
muufl_hsi_reshaped = muufl_hsi_data.reshape(-1, muufl_hsi_data.shape[2])  # (325*220, 64)
muufl_lidar_reshaped = muufl_lidar_data.flatten().reshape(-1, 1)  # (325*220, 1)

In [ ]:
# Standardize the data
scaler_hsi = StandardScaler()
scaler_lidar = StandardScaler()

hsi_scaled = scaler_hsi.fit_transform(hsi_reshaped)
lidar_scaled = scaler_lidar.fit_transform(lidar_reshaped)

# Fit PLS Regression (1 component, since we analyze each band separately)
pls = PLSRegression(n_components=5)
pls.fit(hsi_scaled, lidar_scaled)

# Extract the regression coefficients for each band
pls_coefficients = pls.coef_.flatten()

# Print the PLS regression coefficients
for i, coef in enumerate(pls_coefficients):
    print(f"Band {i+1} PLS coefficient with LiDAR: {coef}")

In [ ]:
from scipy.stats import pearsonr

# Flatten the HSI and LiDAR data to 2D arrays where rows are pixels and columns are channels/bands
hsi_flat = muufl_hsi_data.reshape((-1, muufl_hsi_data.shape[2]))
lidar_flat = muufl_lidar_data.reshape((-1, muufl_lidar_data.shape[2]))

print('hsi_flatshape:',hsi_flat.shape)
print('lidar_flatshape:',lidar_flat.shape)

# Initialize lists to store correlation coefficients for each LiDAR channel
lidar1_correlation_coefficients = []
lidar2_correlation_coefficients = []

# Calculate Pearson correlation for each HSI channel with the first LiDAR channel
for i in range(hsi_flat.shape[1]):
    corr_lidar1, _ = pearsonr(hsi_flat[:, i], lidar_flat[:, 0])
    lidar1_correlation_coefficients.append(round(abs(corr_lidar1),5))

# Calculate Pearson correlation for each HSI channel with the second LiDAR channel
for i in range(hsi_flat.shape[1]):
    corr_lidar2, _ = pearsonr(hsi_flat[:, i], lidar_flat[:, 1])
    lidar2_correlation_coefficients.append(round(abs(corr_lidar2),5))

# Print the correlation coefficients for the first LiDAR channel
print("Correlation coefficients between HSI channels and the first LiDAR channel:")
for idx, corr in enumerate(lidar1_correlation_coefficients):
    #if idx <=10:
      print(f"HSI Band {idx+1} correlation with first LiDAR channel: {corr:.4f}")

# Print the correlation coefficients for the second LiDAR channel
print("\nCorrelation coefficients between HSI channels and the second LiDAR channel:")
for idx, corr in enumerate(lidar2_correlation_coefficients):
    #if idx <=10:
      print(f"HSI Band {idx+1} correlation with second LiDAR channel: {corr:.4f}")


In [ ]:
#Lists to store correlation coefficients for each LiDA#R channel
lidar1_correlations = []
lidar2_correlations = []

# Calculate Pearson correlation for each HSI channel with both LiDAR channels
for i in range(hsi_flat.shape[1]):
    corr_lidar1, _ = pearsonr(hsi_flat[:, i], lidar_flat[:, 0])
    corr_lidar2, _ = pearsonr(hsi_flat[:, i], lidar_flat[:, 1])
    lidar1_correlations.append(round(abs(corr_lidar1),5))
    lidar2_correlations.append(round(abs(corr_lidar2),5))

# Create a DataFrame to store HSI band index and correlation coefficients
correlation_df = pd.DataFrame({
    'HSI Band Index': range(1, hsi_flat.shape[1] + 1),
    'Correlation with LiDAR Channel 1': lidar1_correlations,
    'Correlation with LiDAR Channel 2': lidar2_correlations
})

# Save the DataFrame to a CSV file
correlation_df.to_csv('Muffle_HSI_LiDAR_Correlations.csv', index=False)

# Print out the DataFrame
print(correlation_df)

In [ ]:
import matplotlib.pyplot as plt

# Assuming lidar1_correlations and lidar2_correlations are the lists you've already computed
# And assuming the HSI band indices are simply 1 through the number of bands
hsi_band_indices = range(1, len(lidar1_correlations) + 1)

# Plotting the correlation with the first LiDAR channel
plt.figure(figsize=(8, 4))
plt.plot(hsi_band_indices, lidar1_correlations, marker='o', linestyle='-', color='blue', label='LiDAR Channel ')
plt.title('MUFFL: Pearson  Correlation Coefficients between LiDAR and Each HSI Bands')
plt.xlabel('HSI Band Number')
plt.ylabel('Correlation Coefficient')
#plt.legend()
plt.grid(True)
plt.show()

# # Plotting the correlation with the second LiDAR channel
# plt.figure(figsize=(8, 4))
# #plt.plot(hsi_band_indices, lidar2_correlations, marker='o', linestyle='-', color='blue', label='LiDAR Channel 2')
# plt.plot(hsi_band_indices, lidar2_correlations, marker='o',linestyle='-', color='b')
# plt.title('MUFFL: Pearson  Correlation Coefficients between LiDAR2 and every HSI Bands')
# plt.xlabel('HSI Band Number')
# plt.ylabel('Correlation Coefficient')
# #plt.legend()
# plt.grid(True)
# plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assuming 'band_numbers' and 'correlation_coefficients' are defined as before
plt.figure(figsize=(9,4.5))
plt.plot(hsi_band_indices, lidar1_correlations, marker='o',linestyle='-', color='b')

# Annotate the first 10 bands and then every 5th point with its correlation coefficient
for i, txt in enumerate(lidar1_correlations):
    if i < 2 or i % 10 == 1:  # Annotate if it's among the first 10 bands or divisible by 5
        plt.annotate(f'{txt:.4f}',
                     (hsi_band_indices[i], lidar1_correlations[i]),
                     textcoords="offset points",
                     xytext=(0,10),
                     ha='center')

plt.xlabel('Band Number')
plt.ylabel('Correlation Coefficients with LiDAR')
plt.title('MUUFL Correlation Coefficients for each HSI Band with LiDAR Data')
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'band_numbers' and 'correlation_coefficients' are defined as before
plt.figure(figsize=(9,4.5))
plt.plot(hsi_band_indices, lidar2_correlations, marker='o',linestyle='-', color='b')

# Annotate the first 10 bands and then every 5th point with its correlation coefficient
for i, txt in enumerate(lidar2_correlations):
    if i < 2 or i % 10 == 1:  # Annotate if it's among the first 10 bands or divisible by 5
        plt.annotate(f'{txt:.4f}',
                     (hsi_band_indices[i], lidar2_correlations[i]),
                     textcoords="offset points",
                     xytext=(0,10),
                     ha='center')

plt.xlabel('Band Number')
plt.ylabel('Correlation Coefficients with LiDAR')
plt.title('MUUFL Correlation Coefficients for each HSI Band with LiDAR Data')
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Assuming lidar1_correlations and lidar2_correlations are the lists of correlation coefficients you've computed
# And assuming the HSI band indices are simply 1 through the number of bands
hsi_band_indices = range(1, len(lidar1_correlations) + 1)

# Plotting both sets of correlation coefficients on the same graph
plt.figure(figsize=(9, 5))
plt.plot(hsi_band_indices, lidar1_correlations, marker='o', linestyle='-', color='blue', label='Correlation with LiDAR Channel 1')
plt.plot(hsi_band_indices, lidar2_correlations, marker='x', linestyle='--', color='red', label='Correlation with LiDAR Channel 2')
plt.title('MUUFL Correlation Coefficients of HSI Bands with LiDAR Channels')
plt.xlabel('HSI Band Number')
plt.ylabel('Correlation Coefficient')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Assuming lidar2_correlations is the list of correlation coefficients you've computed
hsi_band_indices = range(1, len(lidar2_correlations) + 1)

# The list of band indices you want to highlight, each with a unique color
highlighted_bands_cross = [4, 32, 43, 23, 6, 5, 18, 53, 34, 10, 63, 12, 16, 30, 50, 33, 44, 7, 0, 55]
highlighted_bands_lrr =  [40, 0, 33, 25, 1, 39, 29, 24, 17, 22, 26, 27, 21, 20, 19, 18, 23, 15, 16, 14]
highlighted_bands_asps = [14, 8, 1, 5, 3, 4, 13, 49, 15, 22, 21, 20, 19, 18, 17, 16, 11, 12, 48, 10]
highlighted_bands_opbs=[57, 28, 38, 2, 63, 56, 62, 16, 61, 33, 58, 60, 55, 59, 54, 53, 52, 0, 51, 37]
highlighted_bands_trc=[25, 22, 8, 39, 33, 14, 21, 5, 42, 6, 41, 44, 28, 27, 0, 1, 34, 31, 17, 19]
highlighted_bands_bsnet=[47, 56, 8, 44, 63, 54, 1, 53, 24, 52, 38, 59, 31, 35, 51, 34, 23, 62, 19,60]

# Define the vertical offset for plotting points that share the same HSI band number
vertical_offset = 0.005

# Define your highlighted bands and their colors
highlighted_bands_info = {
    'CROSS': {'bands': highlighted_bands_cross, 'color': 'blue'},
    'LRR': {'bands': highlighted_bands_lrr, 'color': 'green'},
    'ASPS': {'bands': highlighted_bands_asps, 'color': 'orange'},
    'OPBS': {'bands': highlighted_bands_opbs, 'color': 'purple'},
    'TRC': {'bands': highlighted_bands_trc, 'color': 'cyan'},
    'BSNET': {'bands': highlighted_bands_bsnet, 'color': 'magenta'}
}

# Plotting the correlation with the second LiDAR channel
plt.figure(figsize=(9, 4.5))
plt.plot(hsi_band_indices, lidar2_correlations, linestyle='-', color='grey', alpha=0.5, label='LiDAR Correlation')

# Function to plot highlighted bands with vertical offsets
def plot_highlighted_bands(bands_info):
    for label, info in bands_info.items():
        offsets = np.linspace(-vertical_offset, vertical_offset, len(info['bands']))
        for idx, band in enumerate(info['bands']):
            if band < len(lidar2_correlations):
                plt.plot(band + 1, lidar2_correlations[band] + offsets[idx],
                         marker='o', color=info['color'], label=label if idx == 0 else "")

# Highlight the specified bands with respective colors and labels
plot_highlighted_bands(highlighted_bands_info)

# Add titles and labels
plt.title('MUUFL Correlation Coefficients of HSI Bands with LiDAR Channels')
plt.xlabel('HSI Band Number')
plt.ylabel('Correlation Coefficient')

# Create a custom legend
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.grid(True)
plt.show()



In [ ]:
#Lidar index=2
bands_p9=[4, 32, 43, 23, 6, 5, 18, 53, 34, 10, 63, 12, 16, 30, 50, 33, 44, 7, 0, 55, 47, 13, 49, 24, 2, 27, 41, 61, 36, 59, 40, 48, 46, 3, 58, 8, 14, 51, 26, 62, 17, 28, 29, 11, 54, 42, 31, 22, 35, 38]

In [ ]:
bands_lrr=[40, 0, 33, 25, 1, 39, 29, 24, 17, 22, 26, 27, 21, 20, 19, 18, 23, 15, 16, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 28, 31, 30, 47, 60, 59, 58, 57, 56, 55, 53, 52, 51, 50, 49, 48, 46, 45]

In [ ]:
bands_asps=[14, 8, 1, 5, 3, 4, 13, 49, 15, 22, 21, 20, 19, 18, 17, 16, 11, 12, 48, 10, 9, 7, 6, 2, 23, 24, 25, 26, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 0]

In [ ]:
bands_opbs=[57, 28, 38, 2, 63, 56, 62, 16, 61, 33, 58, 60, 55, 59, 54, 53, 52, 0, 51, 37, 50, 45, 20, 44, 47, 10, 43, 49, 46, 48, 1, 42, 34, 31, 3, 41, 36, 40, 25, 39, 35, 4, 32, 13, 30, 29, 14, 26, 5, 12]

In [ ]:
bands_trc=[25, 22, 8, 39, 33, 14, 21, 5, 42, 6, 41, 44, 28, 27, 0, 1, 34, 31, 17, 19, 45, 40, 15, 48, 52, 61, 60, 59, 58, 57, 56, 54, 53, 55, 63, 51, 50, 62, 23, 7, 3, 12, 24, 11, 43, 35, 2, 47, 38, 9]

In [ ]:
bands_bsnet=[47, 56, 8, 44, 63, 54, 1, 53, 24, 52, 38, 59, 31, 35, 51, 34, 23, 62, 19, 60, 58, 43, 7, 17, 26, 30, 57, 48, 4, 0, 14, 50, 3, 12, 29, 22, 49, 33, 36, 46, 41, 21, 32, 61, 10, 5, 15, 42, 40, 28]

In [ ]:
# Sample lists of highlighted bands for each method (replace with your actual data)

# The list of band indices you want to highlight, each with a unique color
highlighted_bands_cross = bands_p9
highlighted_bands_lrr =  bands_lrr
highlighted_bands_asps = bands_asps
highlighted_bands_opbs=bands_opbs
highlighted_bands_trc=bands_trc
highlighted_bands_bsnet=bands_bsnet
print(len(highlighted_bands_cross))
# Dictionary to store the count of points greater than 0.15 for each method
points_count = {
    'CROSS': sum(1 for band in highlighted_bands_cross if lidar2_correlations[band] > 0.0395),
    'LRR': sum(1 for band in highlighted_bands_lrr if lidar2_correlations[band] > 0.04),
    'ASPS': sum(1 for band in highlighted_bands_asps if lidar2_correlations[band] > 0.04),
    'OPBS': sum(1 for band in highlighted_bands_opbs if lidar2_correlations[band] > 0.04),
    'TRC': sum(1 for band in highlighted_bands_trc if lidar2_correlations[band] > 0.0395),
    'BSNET': sum(1 for band in highlighted_bands_bsnet if lidar2_correlations[band] > 0.04)

}

# Now, print the count for each method
for method, count in points_count.items():
    print(f"{method} contains {count} points with correlation coefficients less than 0.23.")
